# PAL case study target analysis
This case study concerns the fluorescence measured from the PAL mutant of a cyanobacterium, which contains only photosystem I (PSI) and photosystem II (PSII). With PSI the difference between open and closed reaction centers (RC) is negligible in the fluorescence, because both the oxidized and reduced RC efficiently quench the excitations. However, with PSII there is a huge difference, with closed RCs exhibiting a huge amount of recombination fluorescence. The measurement and the global analysis are described in (Tian et al. 2013). The simultaneous target analysis of PAL in the open and closed states is based upon the model explained in (van Stokkum 2018).

Tian L, Farooq S, van Amerongen H (2013) Probing the picosecond kinetics of the photosystem II core complex in vivo. Physical Chemistry Chemical Physics 15 (9):3146-3154.

van Stokkum I (2018) Systems biophysics: Global and target analysis of light harvesting and photochemical quenching in vivo. In: Croce R, van Grondelle R, van Amerongen H, van Stokkum I (eds) Light Harvesting in Photosynthesis. CRC Press, Boca Raton, pp 467-482.

### Plotting functions ([`pyglotaran_extras`](https://github.com/s-weigand/pyglotaran-extras/commit/20da3593105fb839f86e668dc12dc9ca87c3b9ce) + `matplotlib`)

In [ ]:
import matplotlib.pyplot as plt
from pyglotaran_extras.plotting.plot_overview import plot_overview
from pyglotaran_extras.plotting.style import PlotStyle

plot_style = PlotStyle()
plt.rc("axes", prop_cycle=plot_style.cycler)
plt.rcParams["figure.figsize"] = (21, 14)

### Analysis functions

In [ ]:
from glotaran.analysis.optimize import optimize
from glotaran.io import load_dataset, load_model, load_parameters
from glotaran.project.scheme import Scheme

# Simultaneous Target Analysis of PAL in the open and in the closed state

### Read PAL data

In [ ]:
from pyglotaran_extras import plot_data_overview

dataset_open = load_dataset(
    "PAL_700_ma_tr2_gain50_10uW_590nmfilter_21C_400nm_AV_bc_sh_sel_620_830.ascii"
)
dataset_closed = load_dataset(
    "PAL_DCMU_80uM_WL_SF_700_ma_tr2_gain50_100uW_590nmfilter_21C_400nm_AV_bc_sh_sel_620_830.ascii"
)
plot_data_overview(dataset_open, nr_of_data_svd_vectors=3, title="PAL open")
plot_data_overview(dataset_closed, nr_of_data_svd_vectors=3, title="PAL closed");

### Load model and parameters

In [ ]:
PAL_open_closed_target_model = load_model("models/PAL-open-closed-target-model.yml")
PAL_open_closed_target_parameters = load_parameters("models/PAL-open-closed-target-parameters.yml")
PAL_open_closed_target_model.validate(parameters=PAL_open_closed_target_parameters)

In [ ]:
PAL_open_closed_target_model

### Create scheme and optimize it

In [ ]:
PAL_open_closed_target_scheme = Scheme(
    PAL_open_closed_target_model,
    PAL_open_closed_target_parameters,
    {"dataset_open": dataset_open, "dataset_closed": dataset_closed},
    maximum_number_function_evaluations=5,
)
PAL_open_closed_target_result = optimize(PAL_open_closed_target_scheme)

In [ ]:
PAL_open_closed_target_result

In [ ]:
PAL_open_closed_target_result.data["dataset_open"]

In [ ]:
compartments = PAL_open_closed_target_scheme.model.initial_concentration["input1"].compartments

PAL_open_closed_target_scheme.model.k_matrix["km1"].matrix_as_markdown(compartments).replace(
    "0.0000e+00", ""
)

In [ ]:
PAL_open_closed_target_scheme.model.k_matrix["km_open"].matrix_as_markdown(compartments).replace(
    "0.0000e+00", ""
)

In [ ]:
PAL_open_closed_target_scheme.model.k_matrix["km_closed"].matrix_as_markdown(compartments).replace(
    "0.0000e+00", ""
)

In [ ]:
PAL_open_closed_target_result.data["dataset_closed"]

In [ ]:
PAL_open_closed_target_result.data["dataset_closed"].component_PS1

In [ ]:
PAL_open_closed_target_result.data["dataset_closed"]

### Result plots

In [ ]:
plot_overview(
    PAL_open_closed_target_result.data["dataset_closed"],
    figure_only=False,
    nr_of_data_svd_vectors=3,
    nr_of_residual_svd_vectors=1,
    linlog=True,
    linthresh=50,
);

In [ ]:
plot_overview(PAL_open_closed_target_result.data["dataset_open"], linlog=False, figure_only=False);

In [ ]:
PAL_open_closed_target_result.optimized_parameters

In [ ]:
from pyglotaran_extras import plot_fitted_traces, select_plot_wavelengths

result1 = PAL_open_closed_target_result.data["dataset_open"]
result2 = PAL_open_closed_target_result.data["dataset_closed"]
allresults = [result1, result2]
wavelengths = select_plot_wavelengths(result1, wavelength_range=(670, 750))
fig3tr, axes = plot_fitted_traces(allresults, wavelengths, linlog=True, linthresh=50)